# Lesson 2: Tool Calling

In [1]:
!pip install llama-index-core
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai

## Setup

In [2]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [3]:
import nest_asyncio
nest_asyncio.apply()

## 1. Define a Simple Tool

In [4]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


## 2. Define an Auto-Retrieval Tool

### Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [24]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["Understanding_Climate_Change.pdf"]).load_data()

In [25]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [26]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: Understanding_Climate_Change.pdf
file_path: Understanding_Climate_Change.pdf
file_type: application/pdf
file_size: 206372
creation_date: 2024-10-24
last_modified_date: 2024-08-20

Understanding Climate Change  
Chapter 1: Introduction to Climate Change  
Climate change refers to significant, long -term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past cent ury, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change.  
Historical Context  
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and  human civilization. 
Most of these climate

In [27]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [29]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of climate change?", 
)

In [30]:
print(str(response))

Climate change results in increased CO2 emissions from the combustion of fossil fuels like coal and oil, leading to a rise in global temperatures. Deforestation releases stored carbon back into the atmosphere, exacerbating the greenhouse effect. Methane emissions from agriculture, particularly livestock, also contribute significantly to climate change due to its potent heat-trapping capability.


In [31]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'Understanding_Climate_Change.pdf', 'file_path': 'Understanding_Climate_Change.pdf', 'file_type': 'application/pdf', 'file_size': 206372, 'creation_date': '2024-10-24', 'last_modified_date': '2024-08-20'}


### Define the Auto-Retrieval Tool

In [32]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [43]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What are the Cultural Transformations  described on page 13?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "cultural transformation", "page_numbers": []}
=== Function Output ===
Promoting a culture of sustainability, resilience, and stewardship can foster long-term climate action. Engaging artists, storytellers, and community leaders in climate conversations can help shift cultural perceptions.


In [34]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '13', 'file_name': 'Understanding_Climate_Change.pdf', 'file_path': 'Understanding_Climate_Change.pdf', 'file_type': 'application/pdf', 'file_size': 206372, 'creation_date': '2024-10-24', 'last_modified_date': '2024-08-20'}


## Let's add some other tools!

In [35]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of Understanding_Climate_Change"
    ),
)

In [38]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What are Innovations mention page 12 ?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "Innovations", "page_numbers": ["12"]}
=== Function Output ===
Emerging technologies such as Artificial Intelligence and Machine Learning, Blockchain for Climate Action, and Bioengineering are considered innovations in the context of climate change. Additionally, policy and governance strategies like Integrated Climate Policies, International Cooperation, and Public-Private Partnerships are also seen as innovative approaches to addressing climate change challenges.


In [39]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '12', 'file_name': 'Understanding_Climate_Change.pdf', 'file_path': 'Understanding_Climate_Change.pdf', 'file_type': 'application/pdf', 'file_size': 206372, 'creation_date': '2024-10-24', 'last_modified_date': '2024-08-20'}


In [42]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper ?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Understanding_Climate_Change"}
=== Function Output ===
Climate change refers to significant, long-term changes in the global climate, encompassing temperature, precipitation, and wind patterns over extended periods. Human activities, particularly the burning of fossil fuels and deforestation, have significantly contributed to climate change. Historical context shows Earth's climate has changed throughout history, with recent changes primarily driven by human activities. Modern observations indicate rapid increases in global temperatures, sea levels, and extreme weather events. Mitigation and adaptation are essential approaches to address climate change, involving reducing greenhouse gas emissions and adjusting practices to minimize damage. Renewable energy sources like solar, wind, and hydroelectric power play a crucial role in reducing emissions. Additionally, sustainable practices in agriculture, forestry, 